<a href="https://colab.research.google.com/github/Sushila-Phogat/CE888/blob/master/Lab5/my_recommender.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [152]:
import pandas as pd
import numpy as np

# Assign spreadsheet filename: file
file = 'https://github.com/Sushila-Phogat/CE888/blob/master/jester-data-1.xls?raw=true'

# Load spreadsheet: xl
xl = pd.ExcelFile(file)

# Load a sheet into a DataFrame by name: df1
df1 = xl.parse(0)



df1.drop(df1.columns[[0]], axis = 1, inplace = True) 

df1.head(5)

,-7.82,8.79,-9.66,-8.16,-7.52,-8.5,-9.85,4.17,-8.98,-4.76,-8.5.1,-6.75,-7.18,8.45,-7.18.1,-7.52.1,-7.43,-9.81,-9.85.1,-9.85.2,-9.37,1.5,-4.37,-9.81.1,-8.5.2,1.12,7.82,2.86,9.13,-7.43.1,2.14,-4.08,-9.08,7.82.1,5.05,4.95,-9.17,-8.4,-8.4.1,-8.4.2,...,8.59,3.59,-6.84,-9.03.1,2.82.1,-1.36,-9.08.2,8.3.1,5.68,-4.81,99,99.1,99.2,99.3,99.4,99.5,99.6,-9.42.1,99.7,99.8,99.9,-7.72,99.10,99.11,99.12,99.13,99.14,99.15,99.16,99.17,2.82.2,99.18,99.19,99.20,99.21,99.22,-5.63,99.23,99.24,99.25
0,4.08,-0.29,6.36,4.37,-2.38,-9.66,-0.73,-5.34,8.88,9.22,6.75,8.64,4.42,7.43,4.56,-0.97,4.66,-0.68,3.30,-1.21,0.87,8.64,8.35,9.17,0.05,7.57,4.71,0.87,-0.39,6.99,6.50,-0.92,7.14,9.03,-1.80,0.73,7.09,3.40,-0.87,7.91,...,-6.70,-3.35,-9.03,4.47,4.08,-3.83,8.74,1.12,0.78,7.52,-5.00,2.77,8.3,7.77,7.33,6.21,7.72,8.98,8.64,8.20,3.93,4.85,4.85,6.07,8.98,4.51,-0.05,3.69,4.56,0.58,2.82,-4.95,-0.29,7.86,-0.19,-2.14,3.06,0.34,-4.32,1.07
1,99.00,99.00,99.00,99.00,9.03,9.27,9.03,9.27,99.00,99.00,7.33,7.57,9.37,6.17,-6.36,-6.89,-7.86,9.03,9.03,9.03,7.28,99.00,8.25,99.00,99.00,7.48,7.28,7.28,8.93,99.00,6.17,7.28,99.00,99.00,8.98,7.33,99.00,6.17,9.08,7.33,...,6.46,7.28,99.00,99.00,7.04,7.28,99.00,7.28,8.25,99.00,99.00,99.00,99.0,99.00,99.00,8.93,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,9.03,99.00,99.00,99.00,9.08,99.00,99.00,99.00,99.00,99.00,99.00
2,99.00,8.35,99.00,99.00,1.80,8.16,-2.82,6.21,99.00,1.84,7.33,6.60,6.31,8.11,-7.23,-6.65,1.17,-6.60,-3.64,-2.09,5.34,99.00,99.00,99.00,99.00,2.91,3.93,6.75,6.60,99.00,6.65,-6.12,99.00,7.57,6.21,6.65,99.00,-8.30,7.18,2.82,...,0.00,-3.69,99.00,99.00,7.82,0.24,99.00,7.28,-2.33,99.00,99.00,99.00,99.0,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,99.00,0.63,99.00,99.00,-2.33,99.00,99.00,99.00,99.00,99.00,0.53,99.00,99.00,99.00,99.00,99.00,99.00
3,8.50,4.61,-4.17,-5.39,1.36,1.60,7.04,4.61,-0.44,5.73,8.25,6.84,-3.93,7.23,-2.33,-9.66,2.72,-1.36,2.57,4.51,8.20,6.12,8.30,-1.26,7.77,1.89,-1.17,5.68,8.45,4.61,8.06,-9.47,7.28,5.68,2.48,3.20,-1.26,6.80,4.51,2.48,...,7.38,6.17,4.71,-2.28,7.38,4.56,7.14,4.22,3.01,3.83,99.00,99.00,99.0,99.00,99.00,99.00,4.13,99.00,99.00,99.00,5.24,5.92,0.87,7.28,3.93,-0.63,6.31,4.71,2.82,2.96,5.19,5.58,4.27,5.19,5.73,1.55,3.11,6.55,1.80,1.60
4,-6.17,-3.54,0.44,-8.50,-7.09,-4.32,-8.69,-0.87,-6.65,-1.80,-6.80,-5.73,-5.00,-8.59,0.49,-8.93,-3.69,-2.18,-2.28,-6.12,-3.01,-0.58,-2.38,-7.77,-2.23,-2.28,-5.24,-5.53,-0.68,-6.94,-3.93,-0.34,-6.60,-5.49,-1.55,-0.49,-6.21,-8.06,-5.39,-7.04,...,-6.65,-2.86,-4.61,-9.42,-7.91,-9.81,-9.42,-8.98,-1.94,-6.99,-9.66,-5.19,-5.0,-4.42,-7.28,-6.50,-4.32,-6.94,-9.85,0.73,-6.21,-2.86,-7.62,-6.12,-0.29,-1.41,-3.93,-1.94,-5.68,-4.71,-3.54,-6.89,-0.68,-2.96,-2.18,-3.35,0.05,-9.08,-5.05,-3.45


In [153]:
print(df1.shape)

(24982, 100)


In [154]:
df1.describe()

,-7.82,8.79,-9.66,-8.16,-7.52,-8.5,-9.85,4.17,-8.98,-4.76,-8.5.1,-6.75,-7.18,8.45,-7.18.1,-7.52.1,-7.43,-9.81,-9.85.1,-9.85.2,-9.37,1.5,-4.37,-9.81.1,-8.5.2,1.12,7.82,2.86,9.13,-7.43.1,2.14,-4.08,-9.08,7.82.1,5.05,4.95,-9.17,-8.4,-8.4.1,-8.4.2,...,8.59,3.59,-6.84,-9.03.1,2.82.1,-1.36,-9.08.2,8.3.1,5.68,-4.81,99,99.1,99.2,99.3,99.4,99.5,99.6,-9.42.1,99.7,99.8,99.9,-7.72,99.10,99.11,99.12,99.13,99.14,99.15,99.16,99.17,2.82.2,99.18,99.19,99.20,99.21,99.22,-5.63,99.23,99.24,99.25
count,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,...,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000,24982.000000
mean,34.403125,27.094822,33.265435,36.246713,0.427554,19.343233,-0.401425,-0.616301,36.230231,19.521734,14.108189,11.446008,-1.754598,11.038428,-1.681795,-3.071944,-1.102866,-0.618783,0.174411,-0.921639,2.336581,14.813327,19.703860,35.005655,16.832158,6.108954,3.492614,5.393518,3.160558,27.449948,3.466016,3.290556,31.893691,14.294510,3.125387,3.381379,31.185722,9.611310,8.527722,11.139653,...,3.146801,3.156651,19.296626,30.080167,3.261116,2.773362,29.077233,2.804765,2.817985,19.206020,64.954139,65.510407,65.229395,63.736625,64.059347,64.782868,63.812364,63.888051,63.074365,63.644528,63.128637,63.176892,62.954916,62.415779,62.220579,61.606388,62.262233,61.751896,62.467802,60.687683,61.464367,60.278340,60.420157,59.058023,58.748286,58.539484,58.019793,57.059971,55.751626,60.136534
std,46.707230,44.228739,46.747682,48.813136,5.472535,37.861118,5.741420,5.008903,48.228481,38.330434,32.641755,29.939061,5.155981,29.526952,5.355083,5.281516,4.532883,5.038917,5.082801,5.216647,6.856343,34.587830,39.683399,48.637314,37.052138,21.689304,7.074646,19.659713,6.401367,44.864960,12.037911,5.717044,47.440359,34.058319,5.704023,4.991227,47.176941,27.674165,26.397604,30.229100,...,9.383144,6.160873,39.239009,46.233763,10.976613,6.659418,45.963762,6.105737,6.823102,38.956481,47.396612,45.921631,46.645242,48.076251,47.520446,46.243434,47.220674,46.803250,47.676129,47.134038,47.002562,47.331148,46.966739,47.622145,47.594367,47.967447,47.183607,47.224378,46.480069,48.088479,47.334898,47.946774,47.360166,48.196506,48.311024,48.130845,48.163962,48.717708,49.237364,47.913859
min,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,...,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000,-9.950000
25%,-0.150000,-2.180000,-1.120000,-3.350000,-3.690000,-0.390000,-5.290000,-4.610000,

In [155]:
arr = df1.values
print(arr.shape)
rated = np.where(arr!=99)
print(len(rated[0]), rated[1].shape)

(24982, 100)
1810381 (1810381,)


In [156]:
print(arr)
print(np.where(arr!=99))

[[ 4.08 -0.29  6.36 ...  0.34 -4.32  1.07]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.    8.35 99.   ... 99.   99.   99.  ]
 ...
 [99.   99.   99.   ... 99.   99.   99.  ]
 [99.   99.   99.   ... 99.   99.   99.  ]
 [ 2.43  2.67 -3.98 ... 99.   99.   99.  ]]
(array([    0,     0,     0, ..., 24981, 24981, 24981]), array([ 0,  1,  2, ..., 69, 70, 86]))


In [0]:

def replace(orig, percentage=0.1):
  """
  Replaces 'percentage'% of the original values in 'orig' with 99's
  :param orig: original data array
  :param percentage: percentage of values to replace (0<percentage<1)
  """
  new_data = orig.copy()
  rated = np.where(arr!=99)
  n_rated = len(rated[0])
  idx = np.random.choice(n_rated, size=int(percentage*n_rated), replace=False)
  new_data[rated[0][idx], rated[1][idx]] = 99
  return new_data, (rated[0][idx], rated[1][idx])

In [0]:
new_arr, idx = replace(arr, 0.1)

In [159]:
arr[idx[0][0], idx[1][0]]

7.62

In [160]:
new_arr[idx[0][0], idx[1][0]]

99.0

In [0]:
import pandas as pd
import numpy as np
from IPython.display import Image
np.set_printoptions(precision = 3)

In [162]:
df1.shape[0]

24982

In [0]:
n_latent_factors = 2

user_ratings = df1.values
# Initialise as random values
latent_user_preferences = np.random.random((df1.shape[0], n_latent_factors))
latent_item_features = np.random.random((df1.shape[1], n_latent_factors))

In [164]:
latent_item_features.shape[0]

100

In [165]:
latent_user_preferences

array([[0.023, 0.684],
       [0.774, 0.853],
       [0.527, 0.967],
       ...,
       [0.9  , 0.004],
       [0.539, 0.186],
       [0.049, 0.212]])

In [0]:
def predict_rating(user_id, item_id):
    """ Predict a rating given a user_id and an item_id.
    """
    user_preference = latent_user_preferences[user_id]
    item_preference = latent_item_features[item_id]
    return user_preference.dot(item_preference)


def train(user_id, item_id, rating, alpha=0.0001):
  prediction_rating = predict_rating(user_id, item_id)
  err =  prediction_rating - rating
  user_pref_values = latent_user_preferences[user_id][:]
  latent_user_preferences[user_id] -= alpha * err * latent_item_features[item_id]
  latent_item_features[item_id] -= alpha * err * user_pref_values
  return err
    

def sgd(iterations=300000):
  for iteration in range(iterations):
    error=[]
    for user_id in range(latent_user_preferences.shape[0]):
      for item_id in range(latent_item_features.shape[0]):
        rating = user_ratings[user_id][item_id]
        if not np.isnan(rating):
          err = train(user_id, item_id, rating)
          error.append(err)
          mse = (np.array(error) ** 2).mean()
          if (iteration % 10000) == 0:
            print(mse)

In [167]:
sgd()

Streaming output truncated to the last 5000 lines.
2602.98460209318
2602.5580705761595
2602.1227653901337
2601.684759414606
2601.254650978579
2600.8226624353792
2600.3960133399733
2599.961460893555
2599.5247607200586
2599.090211762634
2598.653993566223
2598.217743696324
2597.783661173981
2597.3587398632903
2596.9228212159064
2596.489013332689
2596.0625719006016
2595.635333435762
2595.206515247757
2594.7758062328026
2594.3495096384895
2593.915967334726
2593.4835079215836
2593.056169259417
2592.624425658516
2592.1988186409226
2591.767988343205
2591.333455711672
2590.9026719328194
2590.474155450675
2590.0507184695716
2589.6175899154678
2589.1866549667716
2588.7591451321264
2589.958580697897
2591.156735857412
2592.337193598486
2593.5203273515385
2594.707795130291
2595.885558248169
2597.0665432734045
2598.241338800769
2599.4306219179402
2600.6228695336904
2601.8022388055188
2602.992114753857
2604.179133346173
2605.3442657671794
2606.5182620681603
2607.682454042072
2608.859273982868
2610.043

KeyboardInterrupt: ignored

In [168]:

predictions = latent_user_preferences.dot(latent_item_features.T)
predictions

array([[0.363, 0.396, 0.337, ..., 1.028, 0.65 , 0.597],
       [0.801, 1.12 , 1.491, ..., 2.662, 2.094, 1.848],
       [0.807, 1.064, 1.307, ..., 2.579, 1.937, 1.724],
       ...,
       [0.228, 0.476, 0.899, ..., 1.007, 1.015, 0.864],
       [0.23 , 0.385, 0.618, ..., 0.865, 0.77 , 0.666],
       [0.12 , 0.14 , 0.14 , ..., 0.355, 0.24 , 0.218]])

In [0]:
values = [zip(user_ratings[i], predictions[i]) for i in range(predictions.shape[0])]
comparison_data = pd.DataFrame(values)
comparison_data.columns = df1.columns

In [196]:
#Best joke
df_mean=[]
df_mean.append(df1.mean(axis = 0, skipna = True))

type(df_mean)
df_mean[-1].sort_values(ascending=False).head(1)


99.1    65.510407
dtype: float64

In [197]:
#worst joke
df_mean=df1.mean(axis = 0, skipna = True)
df_mean.sort_values(ascending=True).head(1) #53th is wort joke

-7.52.1   -3.071944
dtype: float64